# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 4 Assignment: Classification and Regression Neural Network**

**Student Name: Wei Lu**

# Assignment Instructions

For this assignment you will use the **crx.csv** dataset.  This is a public dataset that can be found [here](https://archive.ics.uci.edu/ml/datasets/credit+approval). You should use the CSV file on my data site, at this location: [crx.csv](https://data.heatonresearch.com/data/t81-558/crx.csv) because it includes column headers.  This is a dataset that is usually used for binary classification. There are 15 attributes, plus a target column that contains only + or -.  Some of the columns have missing values.

For this assignment you will train a neural network and return the predictions.  You will submit these predictions to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

Complete the following tasks:

* Your task is to replace missing values in columns *a2* and *a14* with values estimated by a neural network.
* Your submission file will contrain the same headers as the source CSV: *a1*, *a2*, *s3*, *a4*, *a5*, *a6*, *a7*, *a8*, *a9*, *a10*, *a11*, *a12*, *a13*, *a14*, *a15*, and *a16*.
* You should only need to modify *a2* and *a14*.
* Neural networks can be much more powerful at filling missing variables than median and mean.
* Train two neural networks to predict *a2* and *a14*.  
* The y for training the two nets will be *a2* and *a14*, depending on which you are trying to fill.
* The x for training the two nets will be 's3','a8','a9','a10','a11','a12','a13','a15'.  These are chosen because it is important not to use any columns with missing values, also it could cause unwanted bias if we include the ultimate target (*a15*).
* ONLY predict new values for missing values in *a2* and *a14*.
* You will likely get this small warning:  Warning: The mean of column a14 differs from the solution file by 0.20238937709643778. (might not matter if small)



# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [0]:
import base64
import os
import numpy as np
import pandas as pd
import requests

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process.  Running the following code will map your GDrive to /content/drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls /content/drive/My\ Drive/Colab\ Notebooks

assignment_WeiLu_class1.ipynb
assignment_WeiLu_class2.ipynb
assignment_WeiLu_class3.ipynb
assignment_WeiLu_class4.ipynb
“t81_558_class_01_1_overview.ipynb”的副本
“t81_558_class_01_4_python_files.ipynb”的副本
“t81_558_class_01_5_python_functional.ipynb”的副本
“t81_558_class_02_1_python_pandas.ipynb”的副本
“t81_558_class_02_2_pandas_cat.ipynb”的副本
“t81_558_class_02_3_pandas_grouping.ipynb”的副本
“t81_558_class_03_2_keras.ipynb”的副本
“t81_558_class_04_1_feature_encode.ipynb”的副本


# Assignment #4 Sample Code

The following code provides a starting point for this assignment.

In [0]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping


key = "nFpEY2LlkO1YtRFzGfAdnaL2S1FiaWjD7MPdWB8B"  # This is an example key and will not work.
file='/content/drive/My Drive/Colab Notebooks/assignment_WeiLu_class4.ipynb'


# Begin assignment
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/crx.csv",na_values=['?'])

a9_dummies = pd.get_dummies(df['a9'], prefix='a9')
a10_dummies = pd.get_dummies(df['a10'], prefix='a10')
a12_dummies = pd.get_dummies(df['a12'], prefix='a12')
a13_dummies = pd.get_dummies(df['a13'], prefix='a13')
df = pd.concat([df,a9_dummies,a10_dummies,a12_dummies,a13_dummies], axis=1)


def fill_missing_numeric(df,current,target):
    # training set: not containing missing values
    x_columns = df.columns.drop(['a1','a2','a4', 'a5', 'a6', 'a7','a14','a16', 'a9','a10','a12','a13'])
    x = df[df[current].notnull()][x_columns].values
    y = df[df[current].notnull()][current].values
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
    
    # Build the neural network
    model = Sequential()
    model.add(Dense(10, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dense(10, activation='relu')) # Hidden 2
    model.add(Dense(1)) # Output
    model.compile(loss='mean_squared_error', optimizer='adam')

    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=8, verbose=1, mode='auto', restore_best_weights=True)
    model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

    pred = model.predict(df[df[current].isnull()][x_columns].values)
    df[current].fillna(pd.Series(pred.T.flatten()))

    return df[['a1', 'a2', 's3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16']]

  
df_submit = fill_missing_numeric(df,'a2','a16')
df_submit = fill_missing_numeric(df,'a14','a16')


# Submit
submit(source_file=file,data=df_submit,key=key,no=4)

Train on 508 samples, validate on 170 samples
Epoch 1/1000
508/508 - 0s - loss: 46649.0397 - val_loss: 210397.0536
Epoch 2/1000
508/508 - 0s - loss: 18976.0804 - val_loss: 55612.7182
Epoch 3/1000
508/508 - 0s - loss: 5470.0993 - val_loss: 13938.2125
Epoch 4/1000
508/508 - 0s - loss: 2469.3156 - val_loss: 1556.7755
Epoch 5/1000
508/508 - 0s - loss: 1135.6749 - val_loss: 1265.2247
Epoch 6/1000
508/508 - 0s - loss: 1070.6264 - val_loss: 1589.3396
Epoch 7/1000
508/508 - 0s - loss: 1060.8963 - val_loss: 1651.3397
Epoch 8/1000
508/508 - 0s - loss: 1052.4106 - val_loss: 1579.6718
Epoch 9/1000
508/508 - 0s - loss: 1043.8935 - val_loss: 1480.7548
Epoch 10/1000
508/508 - 0s - loss: 1038.6060 - val_loss: 1339.4861
Epoch 11/1000
508/508 - 0s - loss: 1033.2249 - val_loss: 1343.1990
Epoch 12/1000
508/508 - 0s - loss: 1025.0894 - val_loss: 1432.4129
Epoch 13/1000
Restoring model weights from the end of the best epoch.
508/508 - 0s - loss: 1023.8034 - val_loss: 1420.6046
Epoch 00013: early stopping
Tr